In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# analysis function

def analysis(path_lorentz):
    
    # import csv file for amplitudes
    lorentziana = pd.read_csv(path_lorentz, sep=';')
    
    # convert DataFrames to numpy arrays
    ampl = lorentziana['amplitude'].to_numpy()
    freq = lorentziana['frequency'].to_numpy()

analysis('dati/forzato/lorentziana.csv')